In [1]:
using LowLevelFEM
import LowLevelFEM as FEM

gmsh.initialize()

In [2]:
using SparseArrays, LinearAlgebra

In [3]:
gmsh.open("body2.geo")

Info    : Reading 'body2.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000646263s, CPU 0.000621s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000302923s, CPU 0.000302s)
Info    : Meshing 3D

In [4]:
mat = FEM.material("body", μ=100, ν=0.3)
hooke = FEM.Problem([mat], type=:Solid)
stvenkir = FEM.Problem([mat], type=:StVenantKirchhoff)
neohooke = FEM.Problem([mat], type=:NeoHookeCompressible)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 386)
Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 386)
Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 386)


LowLevelFEM.Problem("body2", :NeoHookeCompressible, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 150.0, 100.0, 216.66666666666666)], 1.0, 3321)

In [5]:
supp = FEM.displacementConstraint("left", ux=0, uy=0, uz=0)
supp2 = FEM.displacementConstraint("right", uz=0)
traction = FEM.load("right", fx=100, fy=-0.1)
bodyforce = FEM.load("body", fx=1)

problem = hooke
q = FEM.solveDisplacement(problem, [traction], [supp])
S = FEM.solveStress(problem, q)

LowLevelFEM.TensorField([[97.78893062450794; 10.617215923497891; … ; 0.4973274783424476; 0.22698276163527886;;], [96.59034722576652; -3.5805596230001084; … ; 0.08383390338929118; -0.5488175555724908;;], [97.16638024454917; -0.6466345845668966; … ; 0.0011288559649065655; -0.07083103498406373;;], [97.40935700484073; -0.15165073724877992; … ; -0.0007391890702717718; -0.0011331401840060096;;], [97.59379793455084; -0.12456093062042946; … ; -6.987023169987929e-5; 0.0005741564493888518;;], [97.75003375274466; -0.1256206452419223; … ; 1.4190318942327348e-6; 6.128770404600914e-5;;], [97.90077022814746; -0.12581053977851592; … ; 8.167107499331179e-7; -1.6899008414839045e-8;;], [98.05083960652254; -0.12579206808662485; … ; 6.555464601394856e-8; -6.271298467197539e-7;;], [98.20084601924394; -0.12578415743979443; … ; -9.175686385911939e-10; -6.715738010143468e-8;;], [98.35084673604015; -0.1257828537842602; … ; -7.099022224766626e-10; -1.1006484612607892e-9;;]  …  [102.98902784610101; -0.01773398410

In [6]:
u1 = FEM.showDoFResults(problem, q, :uvec)
S1 = FEM.showElementResults(problem, S, :s)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


2

In [7]:
problem = stvenkir
problem = neohooke

LowLevelFEM.Problem("body2", :NeoHookeCompressible, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 150.0, 100.0, 216.66666666666666)], 1.0, 3321)

In [8]:
followerLoad = false
loadSteps = 10
firstStep = 0.1
maxIteration = 10
saveSteps = true
saveIterations = false
plotConvergence = true
ε = 1e-3

r0 = FEM.nodePositionVector(problem)
err0 = abs(maximum(r0) - minimum(r0))
f = FEM.loadVector(problem, [traction])
r = []
r1 = []
push!(r, r0)
e = []

for j in range(1, loadSteps)
    fact = j / loadSteps
    err = 1
    i = 0
    while err > ε && i < maxIteration
        i += 1

        Kl = FEM.stiffnessMatrixLinear(problem, r0)
        Knl = FEM.stiffnessMatrixNonLinear(problem, r0)
        if followerLoad == true
            f = FEM.followerLoadVector(problem, r0, [traction])
        end
        fnl = FEM.loadVectorNonLinear(problem, r0)
        K1, f1 = FEM.applyBoundaryConditions(problem, Kl + Knl, fact * f - fnl, [supp, supp2])
        q = FEM.solveDisplacement(K1, f1)
        r0 += q
        if saveIterations == true
            push!(r, r0)
        end
        err = maximum(abs.(q)) / err0
        if plotConvergence == true
            append!(e, err)
        end
    end
    if saveSteps == true
        push!(r, r0)
    end
end
if saveIterations == true || saveSteps == true
    n = length(r)
    r1 = zeros(length(r0), n)
    for i in 1:n
        r1[:, i] = r[i]
    end
else
    r1 = zeros(length(r0), 1)
    r1 = r0
end

MethodError: MethodError: no method matching iterate(::LowLevelFEM.VectorField)
The function `iterate` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  iterate(!Matched::CompositeException, Any...)
   @ Base task.jl:55
  iterate(!Matched::Core.MethodMatch, !Matched::Int64)
   @ Base deprecated.jl:293
  iterate(!Matched::Core.MethodMatch)
   @ Base deprecated.jl:293
  ...


In [9]:
using Plots
plot(log10.(e))

UndefVarError: UndefVarError: `e` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [10]:
u1 = similar(r1)
r0 = FEM.nodePositionVector(problem)
for i in 1:size(r1, 2)
    u1[:, i] = r1[:, i] - r0
end

u2 = FEM.showDoFResults(problem, u1, :uvec, t=1:size(u1, 2), visible=true)

UndefVarError: UndefVarError: `r1` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [11]:
r = FEM.nodePositionVector(problem)
ux(x, y, z) = 0.4x
uy(x, y, z) = 0.5y
uz(x, y, z) = 0.6z
q0 = FEM.field("body", fx=ux, fy=uy, fz=uz)
q = FEM.vectorField(problem, [q0])
F = FEM.deformationGradient(problem, q)

ErrorException: deformationGradient: dimension is 3, problem type is NeoHookeCompressible. (They must be 3 and :Solid.)

In [12]:
reshape(F.A[1][1:9], 3, 3)

UndefVarError: UndefVarError: `F` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [13]:
E = (F' * F - FEM.unitTensor(F)) / 2

reshape(E.A[1][1:9], 3, 3)

UndefVarError: UndefVarError: `F` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [14]:
e = (FEM.unitTensor(F) - inv(F * F')) / 2

reshape(e.A[1][1:9], 3, 3)

UndefVarError: UndefVarError: `F` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [15]:
e = inv(F') * E * inv(F)

reshape(e.A[1][1:9], 3, 3)

UndefVarError: UndefVarError: `F` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [16]:
U = sqrt(F' * F)

reshape(U.A[1][1:9], 3, 3)

UndefVarError: UndefVarError: `F` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [17]:
Ex = 10
νxy = 0.3
λ = Ex * νxy / ((1 + νxy) * (1 - 2νxy))
μ = Ex / (2 * (1 + νxy))
I3 = FEM.unitTensor(F)
iC = inv(F' * F)
J1 = det(F)
SII = μ * (I3 - iC) + λ * log(J1) / J1 * I3

reshape(SII.A[1][1:9], 3, 3)

UndefVarError: UndefVarError: `F` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [18]:
function probe(A, x, y, z)
    elementTag, elementType, nodeTags, u, v, w = gmsh.model.mesh.getElementByCoordinates(x, y, z, 3, false)
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elementType)
    comp, fun, ori = gmsh.model.mesh.getBasisFunctions(elementType, [u, v, w], "Lagrange")
    SS = [0.0, 0, 0, 0, 0, 0, 0, 0, 0]
    if isa(A, FEM.TensorField)
        ind = findfirst(i -> i == elementTag, SII.numElem)
        for i in range(1, 9)
            SS[i] = fun' * SII.A[ind][i:9:9numNodes, 1]
        end
    elseif isa(A, Matrix)
        for i in range(1, 9)
            SS[i] = fun' * A[9nodeTags.-(9-i), 1]
        end
    end
    return reshape(SS, 3, 3)
end

probe (generic function with 1 method)

In [19]:
SII2 = FEM.elementsToNodes(problem, SII)

UndefVarError: UndefVarError: `SII` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [20]:
probe(SII2, 0, 0, 0)

UndefVarError: UndefVarError: `SII2` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [21]:
field = FEM.field("body", f=2)
sc = FEM.ScalarField(problem, [field])

LowLevelFEM.ScalarField([[2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;]  …  [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;], [2.0; 2.0; … ; 2.0; 2.0;;]], Matrix{Float64}(undef, 0, 0), Float64[], [473, 474, 475, 476, 477, 478, 479, 480, 481, 482  …  783, 784, 785, 786, 787, 788, 789, 790, 791, 792], 1, :scalarInElements)

In [22]:
I3 = FEM.unitTensor(F)
J = det(F)

UndefVarError: UndefVarError: `F` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [23]:
sc * I3

UndefVarError: UndefVarError: `I3` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [24]:
I3 - I3

UndefVarError: UndefVarError: `I3` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [25]:
σ = F * SII * F' / det(F)

reshape(σ.A[1][1:9], 3, 3)

UndefVarError: UndefVarError: `F` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [26]:
J = 0.5 * 0.5 * 0.5
μ / J * (J^(2 / 3) - 1) + λ / J * log(J)

-119.05114807753088

In [27]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


In [28]:
gmsh.finalize()